# Data Wrangling

This first capstone is intended to forecast the consumption of meat globally over the next 10 years. We will look for patterns in potential social, economic, and environmental indicators that could be predictors of consumption. 

When downloading the value, we saw visually that different datasets had different numbers of null values. We'll want to inspect each dataset and treat our nulls so that we can have a more reasonably sized dataframe.

In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from lib.sb_utils import save_file

## Population

We start by investigating the population dataset. Intuitively, we will want to prioritize the most populous countries since this likely corresponds with richer information about consumption trends.

In [70]:
pop = pd.read_csv('new data/population.csv')
pop.head()

,Domain,Area,Year,Unit,Value
0,Annual population,Afghanistan,1991,1000 persons,13299.017
1,Annual population,Afghanistan,1992,1000 persons,14485.546
2,Annual population,Afghanistan,1993,1000 persons,15816.603
3,Annual population,Afghanistan,1994,1000 persons,17075.727
4,Annual population,Afghanistan,1995,1000 persons,18110.657


In [71]:
#Save the population unit for later, we can then drop that column
pop_unit=1000

In [72]:
#Clean our dataset of nulls and unnecessary columns
pop.isna().sum()

Domain      0
Area        0
Year        0
Unit      475
Value     475
dtype: int64

In [73]:
pop.dropna(inplace=True)
pop.reset_index(drop=True, inplace=True)
pop.drop(labels=['Domain','Unit'], axis=1, inplace=True)

In [74]:
#Get the 100 topmost countries by population
top100 = pop.sort_values('Value',ascending=False).Area.unique()[:100]
poptop100 = pop[pop.Area.isin(top100)]

## Food Balances
Our food balance sheet tells us the production quantity of each meat type in unit tons.

In [75]:
food = pd.read_csv('new data/food_balances.csv')
food.head()

,Domain,Area,Element,Item,Year,Unit,Value
0,"Food Balances (-2013, old methodology and popu...",Afghanistan,Production,Bovine Meat,1991,1000 tonnes,86.0
1,"Food Balances (-2013, old methodology and popu...",Afghanistan,Production,Bovine Meat,1992,1000 tonnes,86.0
2,"Food Balances (-2013, old methodology and popu...",Afghanistan,Production,Bovine Meat,1993,1000 tonnes,97.0
3,"Food Balances (-2013, old methodology and popu...",Afghanistan,Production,Bovine Meat,1994,1000 tonnes,113.0
4,"Food Balances (-2013, old methodology and popu...",Afghanistan,Production,Bovine Meat,1995,1000 tonnes,130.0


In [76]:
food.shape, food.isna().sum()

((20082, 7),
 Domain        0
 Area          0
 Element       0
 Item          0
 Year          0
 Unit       1545
 Value      1545
 dtype: int64)

In [77]:
#Next we pivot our data frame so we have columns for each type of meat
food = pd.pivot_table(food, values='Value',index=['Area','Year'], columns='Item').reset_index()
food.head()

Item,Area,Year,Bovine Meat,Mutton & Goat Meat,Pigmeat,Poultry Meat
0,Afghanistan,1991,86.0,137.0,NaN,12.0
1,Afghanistan,1992,86.0,133.0,NaN,12.0
2,Afghanistan,1993,97.0,132.0,NaN,12.0
3,Afghanistan,1994,113.0,134.0,NaN,12.0
4,Afghanistan,1995,130.0,134.0,NaN,12.0


In [78]:
food.shape, food.isna().sum()

((4768, 6),
 Item
 Area                    0
 Year                    0
 Bovine Meat            28
 Mutton & Goat Meat    140
 Pigmeat               316
 Poultry Meat           51
 dtype: int64)

The column we care about most here is Value, so if it's null, we can drop those rows.

In [9]:
food.dropna(inplace=True)
food.reset_index(drop=True, inplace=True)
food.head()

,Area,Item,Year,Value
0,Afghanistan,Bovine Meat,1991,86.0
1,Afghanistan,Bovine Meat,1992,86.0
2,Afghanistan,Bovine Meat,1993,97.0
3,Afghanistan,Bovine Meat,1994,113.0
4,Afghanistan,Bovine Meat,1995,130.0


In [81]:
food[food['Bovine Meat']==0].Area.unique()

array(['Antigua and Barbuda', 'Bahamas', 'Barbados', 'Bermuda',
       'Cabo Verde', 'Dominica', 'French Polynesia', 'Grenada', 'Kuwait',
       'Netherlands Antilles (former)', 'Saint Kitts and Nevis',
       'Saint Lucia', 'Saint Vincent and the Grenadines',
       'Sao Tome and Principe', 'Trinidad and Tobago'], dtype=object)

In [83]:
food[food['Pigmeat'].isnull()].Area.unique()

array(['Afghanistan', 'Bangladesh', 'Djibouti', 'Iraq', 'Jordan',
       'Kuwait', 'Oman', 'Pakistan', 'Saudi Arabia', 'Sudan',
       'Sudan (former)', 'United Arab Emirates', 'Yemen'], dtype=object)

## Prices
Our prices data shows the Producer Price in dollars per ton for each meat.

In [59]:
prices = pd.read_csv('new data/meat_prices.csv')
prices.head()

,Domain,Area,Element,Item,Year,Months,Unit,Value
0,Producer Prices,Afghanistan,Producer Price (USD/tonne),"Meat, cattle",1991,Annual value,NaN,NaN
1,Producer Prices,Afghanistan,Producer Price (USD/tonne),"Meat, chicken",1991,Annual value,NaN,NaN
2,Producer Prices,Afghanistan,Producer Price (USD/tonne),"Meat, goat",1991,Annual value,NaN,NaN
3,Producer Prices,Afghanistan,Producer Price (USD/tonne),"Meat, pig",1991,Annual value,NaN,NaN
4,Producer Prices,Afghanistan,Producer Price (USD/tonne),"Meat, cattle",1992,Annual value,NaN,NaN


In [60]:
prices.isna().sum()

Domain         0
Area           0
Element        0
Item           0
Year           0
Months         0
Unit       14923
Value      14923
dtype: int64

In [55]:
prices[prices.Value==0]

,Domain,Area,Element,Item,Year,Months,Unit,Value


In [61]:
prices = pd.pivot_table(prices, values='Value',index=['Area','Year'], columns='Item').reset_index()
prices.head()

Item,Area,Year,"Meat, cattle","Meat, chicken","Meat, goat","Meat, pig"
0,Albania,1993,3116.2,NaN,4330.1,2502.0
1,Albania,1994,2071.4,NaN,3438.0,2078.8
2,Albania,1995,2481.2,NaN,1723.2,2481.2
3,Albania,1996,2870.8,NaN,1965.3,2679.5
4,Albania,1997,2350.1,NaN,1762.0,2014.3


In [15]:
prices.dropna(inplace=True)
prices.reset_index(drop=True, inplace=True)

In [62]:
prices.isna().sum()

Item
Area                0
Year                0
Meat, cattle      364
Meat, chicken     501
Meat, goat       1312
Meat, pig         634
dtype: int64

In [66]:
#We have a lot of null values for goat meat - this might make our analysis difficult later so we'll drop those rows
prices.drop(labels='Meat, goat',axis=1,inplace=True)
food.drop(labels='Mutton & Goat Meat', axis=1,inplace=True)

KeyError: "['Mutton & Goat Meat'] not found in axis"

In [68]:
top100[:10]

array(['China, mainland', 'India', 'United States of America', 'USSR',
       'Indonesia', 'Pakistan', 'Brazil', 'Nigeria', 'Bangladesh',
       'Russian Federation'], dtype=object)

In [19]:
prices.dropna(inplace=True)
prices.reset_index(drop=True,inplace=True)

## CO2 Emissions
Finally we will look at the CO2 emissions by country.

In [20]:
co2 = pd.read_csv('new data/emissions.csv')
co2.head()

,Domain,Area,Element,Item,Year,Source,Unit,Value
0,Emissions Totals,Afghanistan,Emissions (CO2eq) (AR5),IPCC Agriculture,1991,FAO TIER 1,kilotonnes,8760.3622
1,Emissions Totals,Afghanistan,Emissions (CO2eq) (AR5),IPCC Agriculture,1992,FAO TIER 1,kilotonnes,8786.8525
2,Emissions Totals,Afghanistan,Emissions (CO2eq) (AR5),IPCC Agriculture,1993,FAO TIER 1,kilotonnes,8865.5553
3,Emissions Totals,Afghanistan,Emissions (CO2eq) (AR5),IPCC Agriculture,1994,FAO TIER 1,kilotonnes,8947.8165
4,Emissions Totals,Afghanistan,Emissions (CO2eq) (AR5),IPCC Agriculture,1995,FAO TIER 1,kilotonnes,9407.0224


In [21]:
co2_unit = 'kilotonnes'
co2.isna().sum()

Domain       0
Area         0
Element      0
Item         0
Year         0
Source       0
Unit       431
Value      431
dtype: int64

In [22]:
co2.dropna(inplace=True)
co2.reset_index(drop=True, inplace=True)
co2.drop(labels=['Domain','Element','Item','Unit','Source'],axis=1,inplace=True)
co2.head()

,Area,Year,Value
0,Afghanistan,1991,8760.3622
1,Afghanistan,1992,8786.8525
2,Afghanistan,1993,8865.5553
3,Afghanistan,1994,8947.8165
4,Afghanistan,1995,9407.0224


## Merging
We'll merge our two dataframes together using an inner join, since we're really only interestsed in years and countries where we have both datapoints

In [37]:
merged = pd.merge(poptop100, food, left_on=['Area','Year'], right_on = ['Area','Year'])

In [40]:
merged.rename(columns={'Bovine Meat':'Bovine Production', 'Pigmeat':'Pig Production','Poultry Meat':'Poultry Production','Value':'Pop'},inplace=True)

In [41]:
merged

,Area,Year,Pop,Bovine Production,Pig Production,Poultry Production
0,Afghanistan,2014,33370.794,137.0,0.0,22.0
1,Afghanistan,2015,34413.603,136.0,0.0,24.0
2,Afghanistan,2016,35383.032,135.0,0.0,25.0
3,Afghanistan,2017,36296.113,128.0,0.0,28.0
4,Afghanistan,2018,37171.921,130.0,0.0,29.0
...,...,...,...,...,...,...
2190,Zimbabwe,2014,13586.707,98.0,21.0,58.0
2191,Zimbabwe,2015,13814.629,114.0,21.0,61.0
2192,Zimbabwe,2016,14030.331,111.0,21.0,67.0
2193,Zimbabwe,2017,14236.595,108.0,17.0,69.0


In [50]:
merged[(merged.Area=='United States of America') &(merged.Year>2014)]
prices[(prices.Year>2015)].Area.unique()

array(['Albania', 'Armenia', 'Austria', 'Azerbaijan', 'Belgium',
       'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Cabo Verde',
       'China, mainland', 'Cyprus', 'Denmark', 'Estonia', 'Fiji',
       'Finland', 'France', 'Georgia', 'Germany', 'Greece', 'Grenada',
       'Iceland', 'Kenya', 'Kyrgyzstan', 'Latvia', 'Luxembourg',
       'Malaysia', 'Malta', 'Mauritius', 'Mexico', 'Mozambique',
       'North Macedonia', 'Norway', 'Romania', 'Russian Federation',
       'Rwanda', 'Saint Lucia', 'South Africa', 'Spain', 'Suriname',
       'Thailand', 'Togo', 'Uruguay', 'Viet Nam', 'Zambia'], dtype=object)

In [26]:
#merge the above with price data
merged = pd.merge(prices, merged, left_on=['Area','Year'], right_on = ['Area','Year'])

In [28]:
merged.rename(columns={'Meat, cattle':'Bovine Price','Meat, chicken':'Poultry Price','Meat, pig':'Pig Price'},inplace=True)
merged.head()

,Area,Year,Bovine Price,Poultry Price,Pig Price,Pop,Bovine Production,Pig Production,Poultry Production
0,Argentina,2004,1391.6,1048.2,929.2,38491.972,3024.0,160.0,909.0
1,Argentina,2005,1620.1,916.4,954.3,38892.931,3131.0,185.0,1053.0
2,Argentina,2006,1496.4,739.2,946.0,39289.878,3034.0,230.0,1202.0
3,Argentina,2007,1558.7,929.4,1071.3,39684.295,3224.0,240.0,1288.0
4,Argentina,2008,1741.1,981.3,1281.0,40080.160,3132.0,274.0,1445.0


In [29]:
merged = pd.merge(merged, co2, left_on=['Area','Year'], right_on = ['Area','Year'])
merged.head()

,Area,Year,Bovine Price,Poultry Price,Pig Price,Pop,Bovine Production,Pig Production,Poultry Production,Value
0,Argentina,2004,1391.6,1048.2,929.2,38491.972,3024.0,160.0,909.0,136899.1923
1,Argentina,2005,1620.1,916.4,954.3,38892.931,3131.0,185.0,1053.0,136478.2809
2,Argentina,2006,1496.4,739.2,946.0,39289.878,3034.0,230.0,1202.0,139872.1149
3,Argentina,2007,1558.7,929.4,1071.3,39684.295,3224.0,240.0,1288.0,142662.1991
4,Argentina,2008,1741.1,981.3,1281.0,40080.160,3132.0,274.0,1445.0,140063.2322


In [30]:
merged.rename(columns={'Value':'Emissions'}, inplace=True)

In [31]:
merged.isna().sum()

Area                  0
Year                  0
Bovine Price          0
Poultry Price         0
Pig Price             0
Pop                   0
Bovine Production     0
Pig Production        0
Poultry Production    0
Emissions             0
dtype: int64

In [32]:
merged.shape

(812, 10)

## Save our data

In [33]:
datapath = 'new data' 
save_file(merged, 'merged.csv', datapath)

Writing file.  "new data/merged.csv"
